# <div style="text-align: center">18.335/6.337 Final Project - The L-BFGS algorithm</div>
##  <div style="text-align: center">Large-Scale Problem (using Strong Wolfe Line-Search)</div>
### <div style="text-align: center">Created by Yusu Liu and Simon Batzner</div>

## Import test functions and set up environment

In [28]:
function lbfgs!(F, x0, maxIt, m, save_pos = 1, τgrad=1e-8, verbose = 0)
    #INPUT
    # F: function to be optimized
    # x0: initial guess
    # maxIt: maximum Iteration
    # m: last m input differences and gradient differences are stored
    # τgrad: tolerance for norm of the slope


    #OUTPUT
    #x1: optimized variable
    #f1: function value at x1
    #k iteration number
    n = size(x0, 1)
    pos = zeros(maxIt, n)
    k=0
    n=length(x0)
    Sm=zeros(n,m) #S_k=x_k+1-x_k
    Ym=zeros(n,m) #Y_k=g_k+1-g_k

    f0,g0 = F(x0)
    #use the simplest line search to find step size
    α, f1, g1=strongwolfe(F,-g0,x0,f0,g0)
    x1 = x0 - α.*g0
    k=1
    if save_pos == 1
        pos[1, :] = x0
        pos[2, :] = x1
    end

    while true
       if k>maxIt
           break
       end
       gnorm=norm(g0)
       if gnorm < τgrad
           break
       end
       s0=x1-x0
       y0=g1-g0
       #println("y0=$y0")

       H0=s0'*y0/(y0'*y0) #hessian diagonal satisfying secant condition
       # println(H0)
       # if H0<0
       #     H0=1
       # end
       # println(H0)
       #update Sm and Ym
       if k<=m
           Sm[:,k]=s0
           Ym[:,k]=y0
           p=-approxInvHess(g1,Sm[:,1:k],Ym[:,1:k],H0)
       # only keep m entries in Sm and Ym so purge the old ones
       elseif (k>m)
           Sm[:,1:(m-1)]=Sm[:,2:m]
           Ym[:,1:(m-1)]=Sm[:,2:m]
           Sm[:,m]=s0
           Ym[:,m]=y0
           p=-approxInvHess(g1,Sm,Ym,H0)
       end
       # new direction=p, find new step size
       α, fs, gs=strongwolfe(F,p,x1,f1,g1)
       #update for next iteration
       x0=x1
       g0=g1
       x1=x1+α.*p
       pos[k+1, :] = x1
       f1=fs
       g1=gs
       k=k+1

       if verbose == 1
           println("It=$k,x=$x1")
        end
    end

    k=k-1
    if save_pos == 1
        return x1, f1, k, pos
    else 
        return x1, f1, k
    end
end

lbfgs! (generic function with 4 methods)

In [29]:
function strongwolfe(F,d,x0,fx0,gx0,maxIt=100)
   α_m=20
   α_p=0
   c1=1e-4
   c2=0.9
   α_x=1
   gx0=copy(gx0'*d)
   fxp=copy(fx0)
   gxp=copy(gx0)
   i=1
   α_s=0
   fs=copy(fx0)
   gs=copy(gx0)
   while true
       xx=x0+α_x*d
       fxx,gxx=F(xx)
       fs=copy(fxx)
       gs=copy(gxx)
       gxx=copy(gxx'*d)

       if (fxx>(fx0+c1*α_x*gx0)[1]) || (i>1) & (fxx>=fxp)
           α_s,fs,gs=zoom(F,x0,d,α_p,α_x,fx0,gx0)
           return α_s,fs,gs
       end
       if abs(gxx)<=-c2*(gx0)
           α_s=copy(α_x)
           return α_s,fs,gs
       end
       if gxx>=0
       #if abs.(gxx)[1]>=0 && abs.(gxx)[2]>=0
           α_s,fs,gs=zoom(F,x0,d,α_x,α_p,fx0,gx0)
           return α_s,fs,gs
       end
       α_p=copy(α_x)
       fxp=copy(fxx)
       gxp=copy(gxx)

       if i>maxIt
           α_s=α_x
           return α_s,fs,gs

       end
       r=0.8
       #r=0.8
       α_x=α_x+(α_m-α_x)*r
       i=i+1

   end
   return α_s,fs,gs
end

strongwolfe (generic function with 2 methods)

In [30]:
function zoom(F,x0,d,α_l,α_h,fx0,gx0,maxIt=10)
   c1=1e-4
   c2=0.9
   i=0
   α_s=0
   fs=copy(fx0)
   gs=copy(gx0)
   while true
       α_x=0.5*(α_l+α_h)
       α_s=copy(α_x)
       xx=x0+α_x*d
       fxx,gxx=F(xx)
       fs=copy(fxx)
       gs=copy(gxx)
       gxx=gxx'*d
       xl=x0+α_l*d
       fxl,gxl=F(xl)
       if (fxx>(fx0+c1*α_x*gx0)[1]) || fxx>=fxl
           α_h=copy(α_x)
       else
           if abs(gxx)[1]<=-c2*(gx0)
               α_s=copy(α_x)
               return α_s,fs,gs
           end
           if gxx*(α_h-α_l)[1]>=0
               α_h=copy(α_l)
           end
           α_l=copy(α_x)
       end
       i=i+1
       if i>maxIt
           α_s=copy(α_x)
           return α_s,fs,gs
       end
   end

   return α_s,fs,gs
end

zoom (generic function with 2 methods)

In [31]:
function approxInvHess(g,S,Y,H0,n=2)
    #INPUT

    #g: gradient nx1 vector
    #S: nxk matrixs storing S[i]=x[i+1]-x[i]
    #Y: nxk matrixs storing Y[i]=g[i+1]-g[i]
    #H0: initial hessian diagnol scalar

    #OUTPUT
    # p:  the approximate inverse hessian multiplied by the gradient g
    #     which is the new direction
    #notation follows:
    #https://en.wikipedia.org/wiki/Limited-memory_BFGS

    n,k=size(S)
    rho=zeros(k)
    for i=1:k
        rho[i].=abs(1/(Y[:,i]'*S[:,i]))
    end


    q=zeros(n,k+1)
    r=zeros(n,1)
    α=zeros(k,1)
    β=zeros(k,1)

    q[:,k+1]=g

    for i=k:-1:1
        α[i].=rho[i]*S[:,i]'*q[:,i+1]
        q[:,i].=q[:,i+1]-α[i]*Y[:,i]
    end

    z=zeros(n)
    z.=H0*q[:,1]


    for i=1:k
        β[i].=rho[i]*Y[:,i]'*z
        z.=z+S[:,i]*(α[i]-β[i])
    end

    p=copy(z)

    return p
end

approxInvHess (generic function with 2 methods)

## First Test

In [32]:
n = 1000

##################
include("engvall.jl")
x0 = rand(n); 
tol = 1
m = 2
t = 0.0
tol = 1
bool = false

if n === 1000
    x1, f1, k, pos=lbfgs!(engvall_1000, x0, 1000, m); 
    pos = pos[1:(k+1), :]; 
    t += @elapsed lbfgs!(engvall_1000, x0, 1000, m); 
    
    if norm(f1 - 1108) > tol
        bool = false
    else
        bool = true
    end
    fun = "Engvall with n = $n"
    
    println("Final f was $f1")
elseif n === 5000
    x1, f1, k, pos=lbfgs!(engvall_5000, x0, 1000, m); 
    pos = pos[1:(k+1), :]; 
    t += @elapsed lbfgs!(engvall_5000, x0, 1000, m); 
    
    if norm(f1 - 5548) > tol
        bool = false
    else
        bool = true
    end
    fun = "Engvall with n = $n"
    
    println("Final f was $f1")
end
 
println("\n===============================================================\nResults:\n===============================================================")
println("* Algorithm: L-BFGS w/ StrongWolfe - own implementation")
println(" * Function: $(fun)")
println(" * Minimum: $(f1)")
println(" * Iterations: $(k)")
println(" * Convergence to within $(tol): $(conv)")

Final f was 1108.1947187850126

Results:
* Algorithm: L-BFGS w/ StrongWolfe - own implementation
 * Function: Engvall with n = 1000
 * Minimum: 1108.1947187850126
 * Iterations: 34
 * Convergence to within 1: conv


## Define functions

In [33]:
function engvall_1000(x::Vector)
    # fmin=1108
    F=0.0
    f=zeros(n-1)
    for i=2:n
        f=((x[i-1])^2+(x[i])^2)^2-4*(x[i-1])+3

        F +=f
    end
    return F
end

function engvall_5000(x::Vector)
    # fmin=5548
    F=0.0
    f=zeros(n-1)
    for i=2:n
        f=((x[i-1])^2+(x[i])^2)^2-4*(x[i-1])+3

        F +=f
    end
    return F
end

engvall_5000 (generic function with 1 method)

In [34]:
using Optim, LineSearches

## Optim.jl's default (will not converge)

In [35]:
n = 1000
x0 = rand(n);
result = optimize(x -> engvall_1000(x), x0)

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [0.6293337695370573,0.9028302053334671, ...]
 * Minimizer: [0.6300581739109379,0.8984317807571145, ...]
 * Minimum: 1.632738e+03
 * Iterations: 1000
 * Convergence: false
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: false
   * Reached Maximum Number of Iterations: true
 * Objective Calls: 2184

## Newton's Method from Optim.jl for n = 1000

In [36]:
# ### DID NOT CONVERGE!!! ###
# n = 1000
# ########################################################################################################
# k_tot = 0 
# n_x0 = 1

# for i = 1:n_x0
    
#     x0 = rand(n); 
#     result=optimize(x -> engvall_1000(x), x0, Newton(),Optim.Options(extended_trace=true,show_every=0,store_trace = true))  
#     k_tot += result.iterations
    
# end
# println("Average number of iterations for $(fun) averaged over $(n_x0) iterations: $(k_tot/n_x0)")
# print(result)

## Newton's Method from Optim.jl for n = 5000

In [37]:
# n = 5000
# ########################################################################################################
# k_tot = 0 

# for i = 1:n_x0
    
#     x0 = rand(n); 
#     result=optimize(x -> engvall_5000(x), x0, Newton(),Optim.Options(extended_trace=true,show_every=0,store_trace = true))  
#     k_tot += result.iterations
    
# end
# println("Average number of iterations for $(fun) averaged over $(n_x0) iterations: $(k_tot/n_x0)")
# print(result)

## BFGS from Optim.jl for n = 1000

In [38]:
n_x0 = 1
n = 1000
########################################################################################################
k_tot = 0 

for i = 1:n_x0
    x0 = rand(n); 
    println(1)
    result=optimize(x -> engvall_1000(x), x0, BFGS(),Optim.Options(extended_trace=true, show_every=1, store_trace = true))  
    k_tot += result.iterations
    
end
println("Average number of iterations for $(fun) averaged over $(n_x0) iterations: $(k_tot/n_x0)")
print(result)

1
Average number of iterations for Engvall with n = 1000 averaged over 1 iterations: 31.0
Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [0.982197939559432,0.2027238160074285, ...]
 * Minimizer: [0.901026871530356,0.5458895424144249, ...]
 * Minimum: 1.108195e+03
 * Iterations: 31
 * Convergence: false
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 1.97e-08 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 1.23e-15 
   * |g(x)| < 1.0e-08: false 
     |g(x)| = 2.44e-07 
   * Stopped by an increasing objective: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 65
 * Gradient Calls: 65

## BFGS from Optim.jl for n = 5000

In [39]:
n = 5000
########################################################################################################
k_tot = 0 

for i = 1:n_x0
    
    x0 = rand(n); 
    result=optimize(x -> engvall_5000(x), x0, BFGS(),Optim.Options(extended_trace=true,show_every=0,store_trace = true))  
    k_tot += result.iterations
    
end
println("Average number of iterations for $(fun) averaged over $(n_x0) iterations: $(k_tot/n_x0)")
print(result)

Average number of iterations for Engvall with n = 1000 averaged over 1 iterations: 32.0
Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [0.10229007362775855,0.9727234237857976, ...]
 * Minimizer: [0.9010268569646798,0.5458894413357531, ...]
 * Minimum: 5.548668e+03
 * Iterations: 32
 * Convergence: false
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 3.43e-07 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 9.83e-16 
   * |g(x)| < 1.0e-08: false 
     |g(x)| = 1.35e-06 
   * Stopped by an increasing objective: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 67
 * Gradient Calls: 67

## Our own L-BFGS for n = 1000

In [40]:
include("engvall.jl")
n_x0 = 10
n = 1000
maxIt = 1000
println("\n=========================================================\nResults: \n=========================================================")
k_tot = 0 

for i = 1:n_x0
    x0 = rand(n); 
    x1, f1, k, pos=lbfgs!(engvall_1000, x0, 1000, m); 
    k_tot += k
end
println("Average number of iterations for n = $n, averaged over $(n_x0) iterations: $(k_tot/n_x0)")


Results: 
Average number of iterations for n = 1000, averaged over 10 iterations: 38.3


In [41]:
println("\n=========================================================\nResults: \n=========================================================")
mem_tot = 0 
m = 2
n = 1000

for i = 1:n_x0
    x0 = rand(n); 
    mem = @allocated lbfgs!(engvall_1000, x0, 1000, m)
    mem_tot += mem
end

println("Average memory in [kB] allocated form = $m, averaged over $(n_x0) iterations: $(mem_tot/(n_x0*1000))")



Results: 
Average memory in [kB] allocated form = 2, averaged over 10 iterations: 93924.4656


# Our own L-BFGS for n = 5000

In [25]:
n = 5000
maxIt = 1000
println("\n=========================================================\nResults: \n=========================================================")

k_tot = 0 

for i = 1:n_x0
    x0 = rand(n); 
    x1, f1, k, pos=lbfgs!(engvall_5000, x0, 1000, m); 
    k_tot += k
end
println("Average number of iterations for n = $n, averaged over $(n_x0) iterations: $(k_tot/n_x0)")


Results: 
Average number of iterations for n = 5000, averaged over 10 iterations: 39.1


In [27]:
println("\n=========================================================\nResults: \n=========================================================")
mem_tot = 0 
m = 2
n = 5000

for i = 1:n_x0
    x0 = rand(n); 
    mem = @allocated lbfgs!(engvall_5000, x0, 1000, m)
    mem_tot += mem
end

println("Average memory in [kB] allocated for m = $m, averaged over $(n_x0) iterations: $(mem_tot/(n_x0*1000))")



Results: 
Average memory in [kB] allocated for m = 2, averaged over 10 iterations: 605443.5856
